# Deep Deterministic Policy Gradient: DDPG
This notebook is an implementation of the DDPG algorithm to solve the Reacher environment. You can find
an explanation of DDPG in this [paper](https://arxiv.org/abs/1509.02971)
and an explanation of the Reacher enviornment [here](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Learning-Environment-Examples.md).

## 1. Import all necessary packages
If you have any trouble importing these packages make sure you check the README file and have all the necessary dependencies.

In [1]:
from unityagents import UnityEnvironment
from collections import deque
import numpy as np
import torch
import matplotlib.pyplot as plt
%matplotlib inline
from agents import DDPG

#-------- Setup Notebook Variables ---------#
PATH = "C:\Dev\Python\RL\\Udacity_Continuous_Control"
train = True # True trains new agent, False uses already trained agent
watch_untrained = True
watch_trained = True

## 2. Setup the Environment
Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.
**Note:** `file_name` parameter must match the location of the Unity environment that you downloaded.

In [2]:
env = UnityEnvironment(file_name='C:\Dev\Python\RL\\Udacity_Continuous_Control\Reacher_Windows_x86_64_multiple\Reacher.exe', no_graphics=True)
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]
# reset the environment and get env info to setup Agent
env_info = env.reset(train_mode=True)[brain_name]
action_size = brain.vector_action_space_size
states = env_info.vector_observations
state_size = states.shape[1]


INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


## 3. Watch Untrained Agent
This section shows you what the untrained agents looks like performing in the environment. If you do not want to watch
the untrained agents set `watch_untrained = false` in section 1.

In [ ]:
def watch_untrained(agent, env, brain_name):
    # Setup
    env_info = env.reset(train_mode=False)[brain_name]
    states = env_info.vector_observations
    # Run the simulation
    for t in range(300):
        actions = agent.act(states, agent.epsilon, add_noise=False)
        env_info = env.step(actions)[brain_name]
        next_states, dones = env_info.vector_observations, env_info.local_done
        states = next_states
        if np.any(dones):
            break

if watch_untrained:
    agent = DDPG(state_size=state_size, action_size=action_size, num_agents=len(env_info.agents), epsilon=0, random_seed = 2)
    watch_untrained(agent)



## 4. Setup and Train the Agent
This section trains the agent. If you do not want to train the agent and only wish to see the result of training and view the agent then set
`train = false` in section 1.

In [ ]:
ddpg_agent = DDPG(state_size=state_size, action_size=action_size, num_agents=len(env_info.agents), epsilon=1, random_seed = 2)
if train:
    ddpg_agent.train(env, brain_name)


Episode: 10 	Average Score: 0.31
Episode: 20 	Average Score: 0.55
Episode: 30 	Average Score: 1.14
Episode: 40 	Average Score: 1.58
Episode: 50 	Average Score: 2.26
Episode: 60 	Average Score: 3.45
Episode: 70 	Average Score: 5.03
Episode: 80 	Average Score: 8.71
Episode: 90 	Average Score: 13.44
Episode: 100 	Average Score: 21.11
Episode: 110 	Average Score: 27.24
Episode: 120 	Average Score: 35.34


## 5. View the Results of Training

In [ ]:
def graph_results(scores_filename, avgScores_filename, save_graph = True):
    """ Graph results from training an Agent

    :param filename: file to get scores from
    """
    # Read in results from file
    with open(scores_filename) as f:
        scores = [round(float(score),2) for score in f.read().splitlines()]
    with open(avgScores_filename) as f:
        avg_scores = [round(float(score),2) for score in f.read().splitlines()]

    # Graph results
    fig, ax = plt.subplots()
    ax.set(xlabel="Episode #", ylabel="Score", title="DDPG Agent Learning Curve")
    ax.grid()
    ax.plot(np.arange(len(scores)), scores, label="DDPG Agent Episode Scores")
    ax.plot(np.arange(len(avg_scores)), avg_scores, label="DDPG Agent 100 Episode Average")
    ax.plot(np.arange(len(scores)), np.ones(len(scores))*30, color="black", linestyle="dashed", label="Solved")
    ax.legend()
    # Save graph results
    if save_graph:
        fig.savefig(f'{PATH}\images\DDPG_Agent_Multiple.png')
    plt.show()

graph_results(f'{PATH}\scores\DDPG_Agent_Multiple_Scores.txt',
              f'{PATH}\scores\DDPG_Agent_Multiple_AvgScores.txt')


## 6. Watch the Trained Agent
This section shows you what the trained agents looks like performing in the environment. If you do not want to watch the
trained agents set `watch_trained = false` in section 1.

In [ ]:
def watch_trained(agent, env, brain_name):
    # Setup
    agent.actor_local.load_state_dict(torch.load('DDPG_Agent_Actor_Multiple.pth'))
    agent.critic_local.load_state_dict(torch.load('DDPG_Agent_Critic_Multiple.pth'))
    env_info = env.reset(train_mode=False)[brain_name]
    states = env_info.vector_observations
    # Run the simulation
    for t in range(300):
        actions = agent.act(states, agent.epsilon, add_noise=False)
        env_info = env.step(actions)[brain_name]
        next_states, dones = env_info.vector_observations, env_info.local_done
        states = next_states
        if np.any(dones):
            break

if watch_trained:
    agent = DDPG(state_size=state_size, action_size=action_size, num_agents=len(env_info.agents), epsilon=0, random_seed = 2)
    watch_trained(agent, env, brain_name)